# SamplingExplainer - inner working

In this notebook, we take a look at the inner working of the `SamplingExplainer` to understand how it calculates the SHAP  values. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import shap

## Load dataset

In [2]:
# Load iris dataset

X, y = load_iris(return_X_y=True, as_frame=True)

X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

X_train.shape, X_test.shape

((75, 4), (75, 4))

## Train the model to explain

In [4]:
gbm = RandomForestRegressor(n_estimators=5, random_state=3)

gbm.fit(X_train, y_train)

RandomForestRegressor(n_estimators=5, random_state=3)

In [5]:
gbm.score(X_test, y_test)

0.8975044563279857

## Under the hood of the sampling explainer

In [6]:
# set up an explainer

# pass train set with 3 sample to keep it simple
exp = shap.SamplingExplainer(gbm.predict, X_train.head(3))

In [7]:
exp(X_test.head(2), nsamples=10)

  0%|          | 0/2 [00:00<?, ?it/s]

.values =
array([[ 0.37413579,  0.        ,  0.        ,  0.69253039],
       [ 0.14091004,  0.        ,  0.        , -0.0742431 ]])

.base_values =
0.9333333333333333

.data =
array([[5.8, 2.8, 5.1, 2.4],
       [6. , 2.2, 4. , 1. ]])

In [8]:
exp.shap_values(X_test.head(2), nsamples=10)

  0%|          | 0/2 [00:00<?, ?it/s]

array([[ 0.3178323 ,  0.        ,  0.1348834 ,  0.61395064],
       [ 0.2       ,  0.        , -0.1333333 ,  0.        ]])

In [9]:
X = X_test.head(2).values
X

array([[5.8, 2.8, 5.1, 2.4],
       [6. , 2.2, 4. , 1. ]])

In [10]:
X = X_test.head(1).values
X

array([[5.8, 2.8, 5.1, 2.4]])

In [11]:
from shap.utils._legacy import convert_to_instance, match_instance_to_data

instance = convert_to_instance(X)

instance.x

array([[5.8, 2.8, 5.1, 2.4]])

In [12]:
varyingInds = exp.varying_groups(instance.x)

varyingInds

array([0, 1, 2, 3], dtype=int64)

In [13]:
# divide up the samples among the features for round 1
nsamples_each1 = np.ones(4, dtype=np.int64) * 2 * (400 // (4 * 2))
for i in range((400 % (4 * 2)) // 2):
    nsamples_each1[i] += 2
    
nsamples_each1

array([100, 100, 100, 100], dtype=int64)

In [14]:
phi = np.zeros((exp.P, exp.D))
phi_var = np.zeros((exp.P, exp.D))
X_masked = np.zeros((nsamples_each1.max() * 2, exp.data.data.shape[1]))

In [15]:
X_masked = X_masked[:10 * 2,:]
X_masked

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [16]:
inds = np.arange(X.shape[1])
inds

array([0, 1, 2, 3])

In [17]:
X

array([[5.8, 2.8, 5.1, 2.4]])

In [18]:
j = 2
nsamples = 10
x = instance.x
X = exp.data.data
X_masked = X_masked[:nsamples * 2,:]
for i in range(nsamples):
    np.random.shuffle(inds)
    pos = np.where(inds == j)[0][0]
    rind = np.random.randint(X.shape[0])
    X_masked[i, :] = x
    X_masked[i, inds[pos+1:]] = X[rind, inds[pos+1:]]
    X_masked[-(i+1), :] = x
    X_masked[-(i+1), inds[pos:]] = X[rind, inds[pos:]]

X_masked
# evals = f(X_masked)
# evals_on = evals[:nsamples]
# evals_off = evals[nsamples:][::-1]
# d = evals_on - evals_off

array([[5.8, 2.5, 5.1, 1.1],
       [5.8, 3. , 5.1, 2.4],
       [5.8, 2.8, 5.1, 2.4],
       [5.1, 2.8, 5.1, 1.1],
       [5.1, 2.5, 5.1, 1.1],
       [5.8, 2.5, 5.1, 2.4],
       [5.8, 2.8, 5.1, 2.4],
       [5.8, 2.8, 5.1, 2.4],
       [5.8, 2.8, 5.1, 2.4],
       [5.8, 2.8, 5.1, 1.1],
       [5.8, 2.8, 3. , 1.1],
       [5.8, 2.8, 5.1, 2.4],
       [5.8, 2.8, 3. , 2.4],
       [5.8, 2.8, 3. , 2.4],
       [5.8, 2.5, 3. , 2.4],
       [5.1, 2.5, 3. , 1.1],
       [5.1, 2.8, 3. , 1.1],
       [5.8, 2.8, 3. , 2.4],
       [5.8, 3. , 5.1, 2.4],
       [5.8, 2.5, 3. , 1.1]])

In [19]:
x

array([[5.8, 2.8, 5.1, 2.4]])

In [20]:
evals = gbm.predict(X_masked)
evals

C:\Users\Sole\Documents\Repositories\envs\fsml\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([1.4, 2. , 2. , 1.2, 1.2, 2. , 2. , 2. , 2. , 1.4, 1. , 2. , 1.4,
       1.4, 1.4, 0.8, 0.8, 1.4, 2. , 1. ])

In [21]:
evals[:nsamples]

array([1.4, 2. , 2. , 1.2, 1.2, 2. , 2. , 2. , 2. , 1.4])

In [22]:
evals[nsamples:][::-1]

array([1. , 2. , 1.4, 0.8, 0.8, 1.4, 1.4, 1.4, 2. , 1. ])